# Load data

In [4]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [14]:
# Define relevant variables for the ML task
batch_size = 64

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Define your CNN network

we start by creating a class that inherits the nn.Module class, and then we define the layers and their sequence of execution inside \_\_init\_\_ and forward respectively.

Some things to notice here:

- nn.Conv2d is used to define the convolutional layers. We define the channels they receive and how much should they return along with the kernel size. We start from 1 channels, as we are using Grayscale images
- nn.MaxPool2d is a max-pooling layer that just requires the kernel size and the stride
- nn.Linear is the fully connected layer, and nn.ReLU is the activation function used

In the forward method, we define the sequence, and, before the fully connected layers, we reshape the output to match the input to a fully connected layer

Source: https://blog.paperspace.com/writing-cnns-from-scratch-in-pytorch/


In [48]:
# Creating a CNN class
class ConvNeuralNet(torch.nn.Module):
    #  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = torch.nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.max_pool1 = torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer2 = torch.nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.max_pool2 = torch.nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.relu1 = torch.nn.ReLU()
        self.fc1 = torch.nn.Linear(400, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
        
        out = self.conv_layer2(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.relu1(out)
        out = self.fc1(out)
        return out
    
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device\n")

model = ConvNeuralNet(10).to(device)
print(model)

total_params = sum(p.numel() for p in model.parameters())
print(f"Number of Parameters:{total_params}")

Using cpu device

ConvNeuralNet(
  (conv_layer1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (max_pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv_layer2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (max_pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (fc1): Linear(in_features=400, out_features=10, bias=True)
)
Number of Parameters:5258


In [36]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [49]:
# Define relevant variables for the ML task
num_classes = 10
learning_rate = 0.001
num_epochs = 20

# Set Loss function with criterion
loss_fn = nn.CrossEntropyLoss()
# Set optimizer with optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
            


Epoch 1
-------------------------------
loss: 2.309520  [    0/60000]
loss: 2.205447  [ 6400/60000]
loss: 1.814829  [12800/60000]
loss: 1.026267  [19200/60000]
loss: 0.957518  [25600/60000]
loss: 0.793453  [32000/60000]
loss: 0.603216  [38400/60000]
loss: 0.875839  [44800/60000]
loss: 0.726147  [51200/60000]
loss: 0.822019  [57600/60000]
Test Error: 
 Accuracy: 70.3%, Avg loss: 0.731314 

Epoch 2
-------------------------------
loss: 0.825124  [    0/60000]
loss: 0.682887  [ 6400/60000]
loss: 0.727694  [12800/60000]
loss: 0.578035  [19200/60000]
loss: 0.733218  [25600/60000]
loss: 0.922614  [32000/60000]
loss: 0.606198  [38400/60000]
loss: 0.700576  [44800/60000]
loss: 0.851536  [51200/60000]
loss: 0.596363  [57600/60000]
Test Error: 
 Accuracy: 77.0%, Avg loss: 0.625841 

Epoch 3
-------------------------------
loss: 0.472479  [    0/60000]
loss: 0.424525  [ 6400/60000]
loss: 0.330385  [12800/60000]
loss: 0.420370  [19200/60000]
loss: 0.818670  [25600/60000]
loss: 0.576596  [32000/600

<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Exercise :</h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; Change the out_channels of the first conv2d layer to 4 and the out_channels of the second conv2d layer to 8. It is necessory to adapt the rest of layer to this new size. Check that if the training can be done. </h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; 1. By this modification, how much the number of parameters would be reduced? </h1>
    <p style='text-align: left;'> </p>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; 2. How would this modification has impact on training and convergence? </h1>
    <p style='text-align: left;'> </p>
</div>

In [ ]:
# your code is missing here


<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Exercise :</h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; Remove the first MaxPool2d layer and adapt the size of the rest of layers to be trainable. </h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; 1. By this modification, how the number of parameters would change? Does it mean that MAxPool layer contains trainable parameters?</h1>
    <p style='text-align: left;'> </p>
</div>

In [ ]:
# your code is missing here


<div style="border: solid 3px #000;">
    <h1 style="text-align: center; color:#000; font-family:Georgia; font-size:26px;">Exercise :</h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; By using a print of the out.shape() in the function of forward in Network class, the size of each layer's output can be diplayed (ONLY in the training mode.) </h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; 1. Add another Conv2d with stride=2 and MaxPool2d with stride=2 beween the first MaxPool2d and the second Conv2d layer.</h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; 2. Modify the kenel size to 2 in Conv2d layers. Adapt the rest of layers size to have a trainable network.</h1>
    <h1 style="text-align: left; color:#000; font-family:Courier; font-size:16px;"> &emsp; 3. While another Conv2d layer is added, how is the number of model's parameters changed?.</h1>
    <p style='text-align: left;'> </p>
</div>

In [ ]:
# your code is missing here
